In [1]:
import pandas as pd
import numpy as np

## 1) Only when using ExpoApp data: 
## Convert ExpoApp data to equidistant time series 

#### 1.1) Scenario 1: Further analyses will be performed with the pa_proc pipeline ###

In [ ]:
from pa_proc.expoapp_to_acc import expoapp_to_equidistant

# Create dataframe with equidistant acceleration in three axes
data_eq = expoapp_to_equidistant("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp.csv", 
                                 gap_size=1, time_zone="Europe/Madrid")

# Save dataframe to csv (optional)
data_eq.to_csv("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp_eq.csv", index=False, header=False)

#### 1.2) Scenario 2: Further analyses will be performed with GGIR ###

In [ ]:
from pa_proc.expoapp_to_acc import expoapp_to_actigraph

# Create dataframe with equidistant acceleration in three axes, resembling an Actigraph file (to be used in GGIR)
data_act = expoapp_to_actigraph("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp.csv",  
                                gap_size=1, time_zone="Europe/Madrid")

# Save dataframe to csv (optional)
data_act.to_csv("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp_act.csv", index=False, header=False)

## 2) Convert raw acceleration data to Actigraph counts and ENMO

#### 2.1) Scenario 1: Equidistant ExpoApp data 30 Hz in .csv format

In [ ]:
from pa_proc.acc_to_intermediate import acc_to_intermediate

intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp_eq.csv", 
                                          'expoapp-csv', sampling_freq=30, autocalibration=1, location="lumbar")

#### 2.2) Scenario 2: Actigraph data 30 Hz in .csv format

In [ ]:
from pa_proc.acc_to_intermediate import acc_to_intermediate

intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/MOB033 (2022-08-08)RAW.csv",
                                          'actigraph-csv', sampling_freq=30, autocalibration=1, location="hip")

#### 2.3) Scenario 3: Actigraph data at 30 Hz in .gt3x format

In [ ]:
from pa_proc.acc_to_intermediate import acc_to_intermediate

intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/MOB_033 (2022-08-08).gt3x",
                                          'actigraph-gt3x', sampling_freq=30, autocalibration=1, location="hip")

#### 2.4) Scenario 4: Axivity data at 100 Hz in .cwa format

In [5]:
from pa_proc.acc_to_intermediate import acc_to_intermediate

intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/6017276_belt.cwa",
                                          'axivity-cwa', sampling_freq=100, autocalibration=1, location="lumbar")

c:\Anaconda\Lib\site-packages\skdh\preprocessing\calibrate.py:192: UserWarning: Recalibration not done with 360 - 372 hours due to insufficient non-movement data available
  warn(


#### 2.5) Scenario 5: Matrix data at 100 Hz in .csv format

In [7]:
from pa_proc.acc_to_intermediate import acc_to_intermediate

intermediate_60s, enmo_10s = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/MATA00-2002975-20250312-125129.csv",
                                         device = 'matrix-csv', sampling_freq=100, autocalibration=1, location="wrist")

ERROR:root:Epoch data contains 18 samples, which is less than the 90 minimum required samples
c:\Anaconda\Lib\site-packages\skdh\preprocessing\calibrate.py:163: UserWarning: Less than 72 hours of data (30.666666666666668 hours). No Calibration performed
  warn(


In [9]:
enmo_10s

,enmo,wear_vanhees
2025-03-12 11:52:00,0.764856,1.0
2025-03-12 11:52:10,79.024852,1.0
2025-03-12 11:52:20,44.173621,1.0
2025-03-12 11:52:30,36.095922,1.0
2025-03-12 11:52:40,22.090720,1.0
...,...,...
2025-03-12 12:09:40,34.200497,1.0
2025-03-12 12:09:50,7.533318,1.0
2025-03-12 12:10:00,10.275294,1.0
2025-03-12 12:10:10,0.482260,1.0


## 3) calculate physical activity variables

In [9]:
# Identify different days in data
days_measured = intermediate_60s.index.map(pd.Timestamp.date).unique()
 
# Create results dataframe
results = pd.DataFrame(index=days_measured)
 
# Calculate time in different PA domains (cut-offs used from document Gabriela)
for day_loop in days_measured:
	
	day_data = intermediate_60s[(intermediate_60s.index.date == day_loop)]
	
	results.loc[day_loop, 'weartime_choi'] = np.sum(day_data.wear_choi == 1)
	results.loc[day_loop, 'weartime_vanhees'] = np.sum(day_data.wear_vanhees == 1)
	
	day_data_wear_choi = day_data[day_data.wear_choi == 1]
	day_data_wear_vanhees = day_data[day_data.wear_vanhees == 1]
	
	results.loc[day_loop, 'counts_lpa'] = np.sum((day_data_wear_choi.counts_vm > 200) & (day_data_wear_choi.counts_vm <= 2690))
	results.loc[day_loop, 'counts_mvpa'] = np.sum(day_data_wear_choi.counts_vm > 2690)
	results.loc[day_loop, 'counts_vpa'] = np.sum(day_data_wear_choi.counts_vm > 6166)
 
 
	results.loc[day_loop, 'enmo_lpa'] = np.sum((day_data_wear_vanhees.enmo > 47.4) & (day_data_wear_vanhees.enmo < 69.1))
	results.loc[day_loop, 'enmo_mvpa'] = np.sum(day_data_wear_vanhees.enmo >= 69.1)
	results.loc[day_loop, 'enmo_vpa'] = np.sum(day_data_wear_vanhees.enmo >= 258.7)